In [324]:
# prep imports
from google.cloud import bigquery
import requests
import datetime
from dateutil import parser
import warnings
import pandas as pd
warnings.filterwarnings('ignore')


In [325]:
# Set up variables
project_id    = 'atscale-sales-demo'
dataset_id    = 'mm_dataset'
table_queries = 'twinpines_queries_installer'
table_queries_full = project_id + '.' + dataset_id + '.' + table_queries

hostname = "https://52.249.223.76:10502"
bearer_token = 'eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyIwYTJiMjNmMi00ZmYxLTQ3MmMtNzllZS02ZDg3NWRkMzg1ZWEiOnsicXVlcnlfZGF0YXNldF9hcGlfYWxsb3dlZCI6dHJ1ZSwiZGVzaWduX2NlbnRlcl9hcGlfYWxsb3dlZCI6dHJ1ZX0sImFwaXN1cGVydXNlciI6dHJ1ZSwiYXV0aGVudGljYXRlZCI6dHJ1ZSwiYzk4YzI5NTYtYTJiMC00YjA5LTQ1NzctODQ5ZDBmMDY1YzMxIjp7InF1ZXJ5X2RhdGFzZXRfYXBpX2FsbG93ZWQiOnRydWUsImRlc2lnbl9jZW50ZXJfYXBpX2FsbG93ZWQiOnRydWV9LCJkZWZhdWx0Ijp7InF1ZXJ5X2RhdGFzZXRfYXBpX2FsbG93ZWQiOnRydWUsImRlc2lnbl9jZW50ZXJfYXBpX2FsbG93ZWQiOnRydWV9LCJleHAiOjE3MjkwMDI5NzEsImltcGVyc29uYXRpb25zIjp7Im9yZ19pbXBlcnNvbmF0aW9uIjpudWxsfSwiaXNzdWVfdGltZSI6MTcyODk5OTM3MSwicGVybWlzc2lvbnMiOnsib3JnX3Blcm1pc3Npb25zIjp7ImM5OGMyOTU2LWEyYjAtNGIwOS00NTc3LTg0OWQwZjA2NWMzMSI6eyJhZ2dyZWdhdGVzLm1hbmFnZSI6dHJ1ZSwiYWdncmVnYXRlcy52aWV3Ijp0cnVlLCJkYXRhd2FyZWhvdXNlcy5hZG1pbiI6dHJ1ZSwiZGVzaWduY2VudGVyLnVzZXIiOnRydWUsImdsb2JhbC5wcm9qZWN0LmVkaXQiOnRydWUsImdsb2JhbC5wcm9qZWN0LnB1Ymxpc2giOnRydWUsImdsb2JhbC5wcm9qZWN0LnB1Ymxpc2hlZC5yZWFkIjp0cnVlLCJvYmplY3QuY3JlYXRlIjp0cnVlLCJvcmdhbml6YXRpb24uYWRtaW4iOnRydWUsIm92ZXJyaWRlLmVuZ2luZS5hbmQub3JnYW5pemF0aW9uLnNldHRpbmdzIjp0cnVlLCJxdWVyaWVzLm1hbmFnZSI6dHJ1ZSwicXVlcmllcy52aWV3Ijp0cnVlLCJydW50aW1lLnVzZXIuc2V0dGluZ3MiOnRydWUsInZpZXcuc3VwcG9ydC5sb2dzIjp0cnVlfSwiZGVmYXVsdCI6eyJhY2Nlc3MubWV0cmljcy5kYXNoYm9hcmQiOnRydWUsImFnZ3JlZ2F0ZXMubWFuYWdlIjp0cnVlLCJhZ2dyZWdhdGVzLnZpZXciOnRydWUsImRhdGF3YXJlaG91c2VzLmFkbWluIjp0cnVlLCJkZXNpZ25jZW50ZXIudXNlciI6dHJ1ZSwiZ2xvYmFsLnByb2plY3QuZWRpdCI6dHJ1ZSwiZ2xvYmFsLnByb2plY3QucHVibGlzaCI6dHJ1ZSwiZ2xvYmFsLnByb2plY3QucHVibGlzaGVkLnJlYWQiOnRydWUsIm9iamVjdC5jcmVhdGUiOnRydWUsIm9yZ2FuaXphdGlvbi5hZG1pbiI6dHJ1ZSwib3ZlcnJpZGUuZW5naW5lLmFuZC5vcmdhbml6YXRpb24uc2V0dGluZ3MiOnRydWUsInF1ZXJpZXMubWFuYWdlIjp0cnVlLCJxdWVyaWVzLnZpZXciOnRydWUsInJ1bnRpbWUudXNlci5zZXR0aW5ncyI6dHJ1ZSwidmlldy5zdXBwb3J0LmxvZ3MiOnRydWV9fX0sInN1YiI6ImFkbWluIiwic3VwZXJ1c2VyIjp0cnVlfQ.aBJGOwBin1Kk-QaEGUuuYruO_hJ7Tt7pUeu8GUK905hOykoFWCthLjC6YLWoyGlbT9E2X548kntVoczXwpu8n1Te8TmgOn50TF6yA9zjgzivm3DPdrIcjtFIouQ00M76jqpzhyueHAww3yvzvyEzvytQ2L_BC0f1FLdsxhaGiNYFBNtY4702MMV9rYxUO4Rh24-IEVRT_9Ayl99MtgYJc_8HekvWf1qyOQljw2aNBRa6z31lGGtIBeKjdihVuCdF84khz-Oi1EjdTTXUU_bWdrkiJdRZaChUkIHZ1wMLoljD0G8mjzby2UDvufws_3gXc_nXNFAWYlNezAdCaUNqPA'
full_load_hours = 3600 #max limit from atscale <168
maxQueriesPerCall= 9999

In [326]:
#Create GBQ Client
client = bigquery.Client()

In [327]:
#get last load max datetime
lastload_record = None
sql_query = f'SELECT received_time FROM `{table_queries_full}` order by received_time DESC limit 1;'

query_job = client.query(sql_query)
try:
  results = query_job.result()
  for row in results:
    lastload_record = row
    break
except:
  lastload_record = None


In [329]:
lastload = (datetime.datetime.now() - datetime.timedelta(hours=full_load_hours)).isoformat() +'Z'

if lastload_record:
  lastload = lastload_record['received_time']

'load from: ' + lastload

'load from: 2024-05-18T13:51:02.448161Z'

In [330]:
#load log from atscale
url = hostname + "/monitoring/queries/orgId/default?queryStartTimeFrom="+ lastload +"&maxInboundQueries=" + str(maxQueriesPerCall) + "&includeQueryText=True"
print(url)
headers = {
  'Accept': 'application/zip',
  'Authorization': 'Bearer ' + bearer_token

}

records = []
# Start at Page 1
current_page = 1

# Loop through all the pages
while True:
    response = requests.request("GET", url, headers=headers, verify=False, params={'page': current_page })
    if response.status_code <= 299:
      records += response.json()['response']['data']
    else:
      print(response.text)
      raise SystemExit("Stop notebook due to API error")

    # End loop if the current page we requested is the same as the last page the API returns
    #if current_page == response.json()['totalPages']:
    #    break

    # Increment the page to request
    #current_page =response.json()['totalPages']
    current_page += 1
    break




https://52.249.223.76:10502/monitoring/queries/orgId/default?queryStartTimeFrom=2024-05-18T13:51:02.448161Z&maxInboundQueries=9999&includeQueryText=True


In [331]:
if len(records) <1:
  raise SystemExit("No records to update")

In [332]:
rows_denormalized = []

subquery_fields = [
    'type',
    'query_id',
    'query_text',
    'succeeded',
    'started',
    'subquery_fetch_start',
    'finished',
    'dialect',
    'used_local_cache',
    'is_canary',
    'extra_props',
    'subquery_wait_duration',
    'subquery_exec_duration',
    'subquery_fetch_results_duration',
    'subquery_wall_duration',
    'subquery_wall_running_duration'

  ]

extra_props_fields = [
      'jobId',
      'projectId'
  ]

for record in records:
  if 'subqueries' in record.keys():

    for subquery in record['subqueries']:

      for subquery_field in subquery_fields:
        record['subquery_' + subquery_field] = subquery.get(subquery_field)

        if 'extra_props' in subquery.keys():

          for extra_props_record in subquery['extra_props']:

            for extra_props_field in extra_props_fields:
              try:
                record['subquery_extra_props_'+extra_props_field] = extra_props_record[extra_props_field]
              except:
                pass
      rows_denormalized.append(record)
try:
  rows_denormalized = pd.DataFrame(rows_denormalized)
  rows_denormalized.drop(columns=['subqueries'], inplace=True)
except:
  pass
#rows_denormalized.drop(columns=['aggregate_definition_ids'], inplace=True)
#rows_denormalized.drop(columns=['subquery_succeeded'], inplace=True)
#rows_denormalized.drop(columns=['subquery_used_local_cache'], inplace=True)
#rows_denormalized.drop(columns=['subquery_extra_props'], inplace=True)
#rows_denormalized.drop(columns=['subquery_is_canary'], inplace=True)
print('rows to add: ' + str(len(rows_denormalized.index)))

rows to add: 5695


In [333]:
#Create Big Query Schema
from google.cloud import bigquery
from google.cloud.bigquery.schema import SchemaField
from typing import List

def generate_bigquery_schema(df: pd.DataFrame) -> List[SchemaField]:
    TYPE_MAPPING = {
        "i": "INTEGER",
        "u": "NUMERIC",
        "b": "BOOLEAN",
        "f": "FLOAT",
        "O": "STRING",
        "S": "STRING",
        "U": "STRING",
        "M": "TIMESTAMP",
    }
    schema = []
    for column, dtype in df.dtypes.items():

        val = df[column].iloc[0]

        mode = "REPEATED" if isinstance(val, list) else "NULLABLE"
        try:
          if isinstance(val, dict) or (mode == "REPEATED" and isinstance(val[0], dict)):
              fields = generate_bigquery_schema(pd.json_normalize(val))
          else:
              fields = ()
        except:
          print(column)
        type = "RECORD" if fields else TYPE_MAPPING.get(dtype.kind)
        schema.append(
            SchemaField(
                name=column,
                field_type=type,
                mode=mode,
                fields=fields,
            )
        )
    return schema
bigquery_schema = generate_bigquery_schema(rows_denormalized)


aggregate_definition_ids


[SchemaField('type', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('query_id', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('org_id', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('received_time', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('started_planning', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('finished_planning', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('query_finished', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('succeeded', 'BOOLEAN', 'NULLABLE', None, None, (), None),
 SchemaField('aggregate_definition_ids', 'STRING', 'REPEATED', None, None, (), None),
 SchemaField('query_language', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('subqueries_results_processing_duration', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('query_wall_duration', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('query_pre_planning_duration', 'FLOAT', 'NULLABLE', None, None

In [334]:
#Write query log records to gbq

job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=bigquery_schema,
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_APPEND",
)

job = client.load_table_from_dataframe(

    rows_denormalized, table_queries_full, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

table = client.get_table(table_queries_full)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_queries_full
    )
)


Loaded 5695 rows and 40 columns to atscale-sales-demo.mm_dataset.twinpines_queries_installer
